In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

import gc
import pickle as pickle


from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf

In [2]:
gc.collect()

items           = pd.read_csv('items.csv',usecols=["item_id", "item_category_id"])
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')

In [3]:
sales_train[['day','month', 'year']] = sales_train['date'].str.split('.', expand=True).astype(int)
#sales_train = sales_train[sales_train['year'].isin([2013,2014]) == False]
sales_train = sales_train[sales_train['date_block_num'] > 26]
sales_train = sales_train.set_index('item_id').join(items.set_index('item_id'))
sales_train.reset_index(inplace=True)

In [4]:
max_sales=1000
sums = sales_train.groupby('item_id')['item_cnt_day'].sum().reset_index().rename(columns={"item_cnt_day":"item_total_sales"}).sort_values(by='item_total_sales')

ids_keep = sums[(sums['item_total_sales'] > 0) & (sums['item_total_sales'] < max_sales)]['item_id'].unique()
#ids_keep = sums[(sums['item_total_sales'] > 0)]['item_id'].unique()

In [5]:
train_item_ids = sales_train['item_id'].unique()
#train_item_ids = np.setdiff1d(train_item_ids, ids_reject)
#train_item_ids = ids_keep
train_shop_ids = sales_train['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = sales_train['date_block_num'].unique()

#all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_item_ids = test_item_ids

#all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))
all_shop_ids = test_shop_ids



In [6]:
combinations = []

for dbn in range(np.min(train_blocks), np.max(train_blocks)+1):
    sales = sales_train[sales_train.date_block_num==dbn]
    #item_ids = np.intersect1d(sales.item_id.unique(), test_item_ids)
    item_ids = all_item_ids
    #dbn_combos = list(product(sales.shop_id.unique(), item_ids, [dbn]))
    dbn_combos = list(product(all_shop_ids, item_ids, [dbn]))
    for combo in dbn_combos:
        combinations.append(combo)
        
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['shop_id','item_id','date_block_num'])

In [7]:
ys = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_item_cnt_block"})

training = all_combos.merge(ys, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)


training['shop_item_cnt_block'] = training['shop_item_cnt_block'].clip(0,20).astype('int8')

training = training.set_index('item_id').join(items.set_index('item_id'))
training.reset_index(inplace=True)

for col in ['item_id', 'shop_id', 'item_category_id']:
    training[col] = pd.to_numeric(training[col], downcast='unsigned')

In [8]:
dates = sales_train[['date_block_num', 'month', 'year']].drop_duplicates(['date_block_num', 'month', 'year'])

dates_dict = {}

for index,row in dates.iterrows():
    dates_dict[row['date_block_num']] = {"month": row['month'], "year": row['year']}
    
training['month'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['month']), downcast='unsigned')
training['year'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['year']), downcast='unsigned')



In [9]:
ys = sales_train.groupby(['item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"item_cnt_block"})

training = training.merge(ys, on=['item_id', 'date_block_num'], how='left').fillna(0)

ys = sales_train.groupby(['shop_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_cnt_block"})

training = training.merge(ys, on=['shop_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"category_cnt_block"})


training = training.merge(ys, on=['item_category_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['shop_id', 'item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_category_cnt_block"})

training = training.merge(ys, on=['shop_id', 'item_category_id', 'date_block_num'], how='left').fillna(0)


In [11]:
training['item_cnt_block_mean'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.mean)
training['item_cnt_block_min'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.min)
training['item_cnt_block_max'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.max)
training['item_cnt_block_std'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.std)
training['item_cnt_block_med'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.median)

In [12]:
training['shop_cnt_block_mean'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.mean)
training['shop_cnt_block_min'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.min)
training['shop_cnt_block_max'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.max)
training['shop_cnt_block_std'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.std)
training['shop_cnt_block_med'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.median)

In [13]:
training['category_cnt_block_mean'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.mean)
training['category_cnt_block_min'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.min)
training['category_cnt_block_max'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.max)
training['category_cnt_block_std'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.std)
training['category_cnt_block_med'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.median)

In [14]:
training['shop_category_cnt_block_mean'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.mean)
training['shop_category_cnt_block_min'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.min)
training['shop_category_cnt_block_max'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.max)
training['shop_category_cnt_block_std'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.std)
training['shop_category_cnt_block_med'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.median)

In [15]:
training['shop_item_cnt_block_mean'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.mean)
training['shop_item_cnt_block_min'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.min)
training['shop_item_cnt_block_max'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.max)
training['shop_item_cnt_block_std'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.std)
training['shop_item_cnt_block_med'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.median)

In [10]:
#https://maxhalford.github.io/blog/target-encoding-done-the-right-way/
#https://www.kaggle.com/vprokopev/mean-likelihood-encodings-a-comprehensive-study

from sklearn.model_selection import StratifiedKFold
columns = ["item_id", "shop_id", "item_category_id"]



y_train = training["shop_item_cnt_block"].values
folds = KFold(n_splits = 5, shuffle=True).split(training)

i=1
for in_fold_index, out_of_fold_index in folds:
    print("fold", i)
    #print(np.intersect1d(training.loc[in_fold_index]["shop_id"].unique(), training.loc[out_of_fold_index]["shop_id"].unique()))
    #print(len(in_fold_index))
    for column in columns:
        means = training.iloc[in_fold_index].groupby(column)['shop_item_cnt_block'].mean()
            #x_validation[column + "_mean_target"] = means\
        name = column + '_mean_encoding'
        training.loc[out_of_fold_index,name] = training.loc[out_of_fold_index][column].map(means)
    i+=1

fold 1
fold 2
fold 3
fold 4
fold 5


In [11]:
training.fillna(0,inplace=True)

In [12]:

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
training.sample(10)

,item_id,shop_id,date_block_num,shop_item_cnt_block,item_category_id,month,year,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,item_id_mean_encoding,shop_id_mean_encoding,item_category_id_mean_encoding
1287666,18686,50,29,0,55,6,2015,3.0,900.0,6017.0,109.0,0.079498,0.170318,0.192586
92266,1571,50,33,0,23,10,2015,18.0,949.0,2203.0,46.0,0.482759,0.167739,0.405845
1031671,15069,5,31,0,28,8,2015,0.0,1294.0,1722.0,28.0,2.053571,0.179963,0.703351
1094527,15760,55,27,0,40,4,2015,4.0,3422.0,10683.0,0.0,0.096491,0.191819,0.195860
1245880,18045,44,33,0,70,10,2015,12.0,779.0,1960.0,13.0,0.384937,0.131672,0.318802
553876,8219,57,28,0,55,5,2015,0.0,2408.0,6353.0,204.0,0.259414,0.401575,0.195230
1224284,17734,15,32,0,43,9,2015,2.0,1267.0,188.0,0.0,0.029661,0.211569,0.051144
980737,14262,52,29,0,38,6,2015,3.0,820.0,640.0,4.0,0.110204,0.157014,0.155497
768881,11520,16,28,0,69,5,2015,0.0,1187.0,630.0,9.0,0.024896,0.189311,0.164170
1418978,20744,31,28,0,72,5,2015,0.0,6327.0,1195.0,90.0,0.034335,0.886722,0.134023


In [14]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'shop_item_cnt_block',
       'item_category_id', 'month', 'year', 'item_cnt_block',
       'shop_cnt_block', 'category_cnt_block', 'shop_category_cnt_block',
       'item_id_mean_encoding', 'shop_id_mean_encoding',
       'item_category_id_mean_encoding'], dtype=object)

In [15]:
all_features = [
    
   'item_cnt_block',
       'shop_cnt_block', 'category_cnt_block', 'shop_category_cnt_block',
       'item_id_mean_encoding', 'shop_id_mean_encoding',
       'item_category_id_mean_encoding'
    
]



features = [
    
     'item_cnt_block',
       'shop_cnt_block', 'category_cnt_block', 'shop_category_cnt_block',
      
     'item_id_mean_encoding', 'shop_id_mean_encoding',
       'item_category_id_mean_encoding'
]

In [16]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler 


training[all_features] = StandardScaler().fit_transform(training[all_features])



In [17]:
training[all_features] = training[all_features].apply(pd.to_numeric, downcast='float')

In [18]:
training[features].sample(10)

,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,item_id_mean_encoding,shop_id_mean_encoding,item_category_id_mean_encoding
317517,-0.126029,-0.270513,0.837917,0.042962,-0.330579,-0.421868,-0.110983
698169,0.053213,0.055143,-0.016453,0.104175,-0.150287,0.085138,-0.152492
7197,-0.151635,-0.883019,-1.027243,-0.514754,-0.414469,-0.961253,-0.568521
797165,1.384725,1.072436,1.074738,2.239818,0.590820,1.782804,-0.103894
867193,-0.062014,-0.672256,0.612984,0.396635,0.402974,-0.610801,1.880439
107302,-0.138832,-0.093989,-0.715653,-0.514754,-0.367136,0.181718,-0.529466
325658,-0.151635,-0.609103,0.900485,0.070168,4.179259,-0.612941,1.880439
19923,-0.126029,-0.349644,-0.878330,-0.507952,-0.390888,-0.236768,-0.213907
182516,-0.074817,-0.762800,0.077087,-0.188286,0.637688,-0.618822,0.497140
678764,-0.087620,-0.024749,-0.954038,-0.514754,-0.296837,0.053526,-0.509674


In [21]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3).fit(training[features])
print(pca.explained_variance_ratio_) 

[0.33439728 0.31461756 0.17791174]


In [23]:
training_pca = pca.transform(training[features])

In [25]:
np.array(training_pca).shape

(1499400, 3)

In [30]:

  
        

import multiprocessing as mp

import importlib
import build_sample
importlib.reload(build_sample)

from build_sample import build_sample_f

if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    res = [pool.apply_async(build_sample_f,args=[window, training, features]) for window in windows]
    pool.close()
    pool.join()
    
lstm_data = []
lstm_y = []

for result in res:
    for idx, sample in enumerate(result.get()[0]):
        lstm_data.append(sample)
        lstm_y.append(result.get()[1][idx])

[27, 28, 29, 30, 31, 32, 33]


In [31]:
small_data = np.array(lstm_data)
small_y = np.array(lstm_y)

In [31]:
print(len(lstm_y))

print(len([y for y in lstm_y if y == 0]))

zeros_indices = {}
for idx,y in enumerate(lstm_y):
    
    if y == 0:
        zeros_indices[idx] = True

632988
601517


In [32]:
lstm_data_no_zeros = []
lstm_y_no_zeros = []
for idx,sample in enumerate(lstm_data):
    if idx not in zeros_indices:
        lstm_data_no_zeros.append(sample)
        lstm_y_no_zeros.append(lstm_y[idx])

In [33]:
lstm_zeros = []
for idx,y in enumerate(lstm_y):
    if idx in zeros_indices:
        lstm_zeros.append(idx)

In [34]:
len(lstm_data_no_zeros)

31471

In [35]:
#print(len(lstm_data_no_zeros))

for zero_idx in np.random.choice(lstm_zeros,30000,replace=False):
    lstm_data_no_zeros.append(lstm_data[zero_idx])
    lstm_y_no_zeros.append(lstm_y[zero_idx])

In [36]:
small_data = np.array(lstm_data_no_zeros)
small_y = np.array(lstm_y_no_zeros)

In [32]:
from sklearn.model_selection import train_test_split

train_data, val_data, y_train, y_val = train_test_split(small_data, small_y, test_size=0.1, random_state=42)

In [ ]:
np.array(lstm_data).shape

In [33]:
train_data.shape

(192780, 6, 4)

In [38]:
y_train.shape

(55323, 1)

In [ ]:
np.mean(lstm_y_no_zeros)

In [ ]:
lstm_y_no_zeros

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten,GRU,CuDNNGRU,CuDNNLSTM
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization


#x_train_scaled = MinMaxScaler().fit_transform(x_train[features])


#x_reshaped = np.reshape(x_train_scaled, (x_train_scaled.shape[0], 10, x_train_scaled.shape[1]))
    
#x_val_scaled_reshaped = np.reshape(x_val_scaled, (x_val_scaled.shape[0], 1, x_val_scaled.shape[1]))




dropout=0.2

my_model = Sequential()
#bi directional?
#my_model.add(CuDNNLSTM(units = 32,\
                # input_shape = (small_data.shape[1],len(features))))
#my_model.add(BatchNormalization())
#my_model.add(CuDNNLSTM(units = 32,\
                 #input_shape = (small_data.shape[1],len(features)), return_sequences=True))
#my_model.add(BatchNormalization())
#my_model.add(Dropout(dropout))
#my_model.add(GRU(use_bias = True,units = 8, dropout=dropout,recurrent_dropout=dropout,input_shape = (small_data.shape[1],len(features))))
#my_model.add(BatchNormalization())
#my_model.add(Dropout(dropout))
my_model.add(LSTM(use_bias = True,unit_forget_bias=True,units = 1, dropout=dropout,\
                  recurrent_dropout=dropout,input_shape = (small_data.shape[1],len(features))))
my_model.add(BatchNormalization())
my_model.add(Dropout(dropout))
#my_model.add(CuDNNLSTM(units = 8,input_shape = (small_data.shape[1],len(features)), return_sequences=True))
#my_model.add(CuDNNLSTM(units = 16,input_shape = (small_data.shape[1],len(features))))

#my_model.add(BatchNormalization())
#my_model.add(LSTM(use_bias = True,unit_forget_bias=True,units = 4, dropout=dropout,recurrent_dropout=dropout,input_shape = (small_data.shape[1],len(features))))
#my_model.add(Dropout(dropout))
#my_model.add(BatchNormalization())
my_model.add(Dense(1))

my_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
my_model.summary()


#lstmd_dataa = np.array(np.array(lstm_data)).reshape(271148,8,51)
#print(lstmd_dataa.shape)
#print(lstmd_dataa)
#lstm_yy = np.array([np.array([y]) for y in lstm_y[0:100]])
#print(lstm_yy.shape)
#print(lstm_yy)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=2, verbose=0)
]

history = my_model.fit(train_data, y_train, batch_size=32, epochs=100,
                      validation_data=(val_data,y_val), callbacks=callbacks
                      )


import matplotlib.pyplot as plt


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

import math
print("best rmse val:", math.sqrt(my_model.history.history['val_mean_squared_error'][-1]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 1)                 24        
_________________________________________________________________
batch_normalization_9 (Batch (None, 1)                 4         
_________________________________________________________________
dropout_10 (Dropout)         (None, 1)                 0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 2         
Total params: 30
Trainable params: 28
Non-trainable params: 2
_________________________________________________________________
Train on 192780 samples, validate on 21420 samples
Epoch 1/100
192780/192780 [==============================] - 120s 623us/step - loss: 1.2563 - mean_squared_error: 1.2563 - val_loss: 1.0202 - val_mean_squared_error: 1.0202
Epoch 2/100
192780/192780 [===========================

In [40]:
training_test = training[(training['shop_id'].isin(test['shop_id'].unique()))\
                         & (training['item_id'].isin(test['item_id'].unique())) \
                        & (training['date_block_num'].isin(windows[0]))]

In [41]:
a = list(range(0, 235620, 21420))
b = list(range(21420, 257040, 21420))
intervals = list(zip(a,b))[:-1]
intervals

[(0, 21420),
 (21420, 42840),
 (42840, 64260),
 (64260, 85680),
 (85680, 107100),
 (107100, 128520),
 (128520, 149940),
 (149940, 171360),
 (171360, 192780),
 (192780, 214200)]

In [44]:

import multiprocessing as mp

import importlib
import build_test
importlib.reload(build_test)

window_size = len(windows[0])

from build_test import build_test_f

if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    res = [pool.apply_async(build_test_f,args=[interval, test, training_test, features, window_size]) for interval in intervals]
    pool.close()
    pool.join()

(0, 21420)
(21420, 42840)
(42840, 64260)
(64260, 85680)
(85680, 107100)
(107100, 128520)
(128520, 149940)
(149940, 171360)
(171360, 192780)
(192780, 214200)


In [45]:
test_lstm_data = []

for interval in intervals:
    for re in res:
        if interval in re.get():
            for sample in re.get()[interval]:
                test_lstm_data.append(sample)

In [ ]:
test_lstm_data = np.array(test_lstm_data)

In [ ]:
test_lstm_data.shape

In [52]:
preds = my_model.predict(np.array(test_lstm_data),batch_size=len(test_lstm_data))
preds.clip(0,20,out=preds)

array([[0.35118505],
       [0.18858851],
       [0.404051  ],
       ...,
       [0.18795037],
       [0.16009891],
       [0.16940397]], dtype=float32)

In [53]:

print(np.mean(preds))
print(np.max(preds))

0.25428843
20.0


In [54]:
submission = test.loc[:,['ID']]
submission['item_cnt_month'] = preds

submission.to_csv('submission.csv', index=False)

In [ ]:
bestpreds = pd.read_csv('submissionbest.csv')['item_cnt_month']
print(np.mean(bestpreds))
print(np.max(bestpreds))

In [ ]:
lr_preds = pd.read_csv('lr110.csv')['item_cnt_month']
lg_preds = pd.read_csv('lg110.csv')['item_cnt_month']
#cb_preds = pd.read_csv('cb102.csv')['item_cnt_month']


#preds = np.mean(np.array([lr_preds, lg_preds]),axis=0)

preds = (lg_preds * 0.50) + (lr_preds * 0.50)